# Preliminaries

This commands are used in Google Colab:

!pip install deap

from google.colab import drive
drive.mount("/content/gdrive")

Importing python packages and setting display parameters

In [3]:
import math as mt
import random as rnd
import numpy as np
import pandas as pd
import itertools as it

from deap import base, creator, tools

import numba
from numba import jit
import joblib

import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import statistics as stats

In [4]:
%matplotlib inline
#%config InlineBackend.figure_format = "retina"

plt.style.use("default")
plt.style.use("bmh")
# plt.rcParams.update({"figure.autolayout": True})
plt.rcParams["figure.figsize"] = (12, 9)
mpl.rcParams["figure.dpi"] = 100
mpl.rcParams["savefig.dpi"] = 100

pd.set_option("display.latex.repr", True)
pd.set_option("display.latex.longtable", True)

In [ ]:
pickle_dir = "./pickle/"
file_sufix = "C_04"
pickle_dir + file_sufix

Run this when working in Google Colab:

pickle_dir = "/content/gdrive/My Drive/Colab Notebooks/thesis/"
pickle_dir + file_sufix

# Fitness Landscape Definition

In [5]:
# Problem domain
x_min = -15
x_max = 15
y_min = -15
y_max = 15

# Known minimum
x_point = -6.01717
y_point = 9.06022

domain = (x_min, x_max, y_min, y_max)
point = (x_point, y_point)
img_size = (8.5, 4.25)

# Problem definition


@jit
def g_fun(x, y):
    mag = np.sqrt(x ** 2.0 + y ** 2.0)
    val = -(50.0 * np.sinc(mag / np.pi) - mag)
    return val.item()


@jit
def f_fun(x, y):
    x_min = -6.01717
    y_min = 9.06022
    f_min = (
        g_fun(x_min + 11.0, y_min + 9.0)
        + g_fun(x_min - 11.0, y_min - 3.0)
        + g_fun(x_min + 6.0, y_min - 9.0)
    )
    tripsinc = (
        g_fun(x + 11.0, y + 9.0)
        + g_fun(x - 11.0, y - 3.0)
        + g_fun(x + 6.0, y - 9.0)
        - (f_min)
    )
    return tripsinc

In [6]:
@jit
def evaluate(individual):
    x = individual[0]
    y = individual[1]
    fitness = f_fun(x, y)
    return (fitness,)

In [7]:
# Testing the minimum
print(f_fun(-6.01717, 9.06022))

0.0


In [8]:
# Testing the function
print(f_fun(-1.0, -1.0), f_fun(-11.0, -9.0), f_fun(11.0, 3.0), f_fun(-6.0, 9.0))

50.62059878583003 5.177364279021976 6.107247239602234 0.031278340140559635


# Setting up the experiment

## Common parameters

In [9]:
# Algorithm parameters
# Number of replicates, and generations per experiment
rep_end = 40
births_end = 60e3

# Genes
gen_size = 2
# Population size
pop_size_lvl = [20, 160]
# Progeny and parents size ratio to population size
b_ratio_lvl = [0.6, 5]

# Progeny parameters
## Crossover probability per gene
cx_pb_lvl = [0.1, 0.9]
## Mutation probability per gene
mut_pb_lvl = [0.1, 0.9]
## Mutation strength
mut_sig_lvl = [0.5, 7.5]

# Selection by tournament
# Tournament size parent selection
k_par_lvl = [2, 6]
# Tournament size survivor selection
k_sur_lvl = [2, 6]

## Factor levels

In [10]:
factors_levels = [
    ("pop", "Population size", "Integer +", min(pop_size_lvl), max(pop_size_lvl)),
    ("b_ratio", "Progeny-to-pop ratio", "Real +", min(b_ratio_lvl), max(b_ratio_lvl)),
    ("cx_pb", "Crossover prob", "Real [0,1]", min(cx_pb_lvl), max(cx_pb_lvl)),
    ("mut_pb", "Mutation prob", "Real [0,1]", min(mut_pb_lvl), max(mut_pb_lvl)),
    ("mut_sig", "Mutation sigma", "Real +", min(mut_sig_lvl), max(mut_sig_lvl)),
    ("k_par", "Parent tourn size", "Integer +", min(k_par_lvl), max(k_par_lvl)),
    ("k_sur", "Surviv tourn size", "Integer +", min(k_sur_lvl), max(k_sur_lvl)),
]

factors_df = pd.DataFrame(
    factors_levels, columns=["Factor", "Label", "Range", "LowLevel", "HighLevel"]
)
factors_df = factors_df.set_index(["Factor"])

factors_df

,Label,Range,LowLevel,HighLevel
Factor,,,,
pop,Population size,Integer +,20.0,160.0
b_ratio,Progeny-to-pop ratio,Real +,0.6,5.0
cx_pb,Crossover prob,"Real [0,1]",0.1,0.9
mut_pb,Mutation prob,"Real [0,1]",0.1,0.9
mut_sig,Mutation sigma,Real +,0.5,7.5
k_par,Parent tourn size,Integer +,2.0,6.0
k_sur,Surviv tourn size,Integer +,2.0,6.0


## Defining the EA elements

We define that the fitness is related to a minimizing problem, and that each individual is represented with a list of numbers

In [11]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

We start the DEAP toolset. At creation, each individual will have 2 genes of type "float" that are randomly initialized in the range [-15; 15].

In [12]:
toolbox = base.Toolbox()

In [13]:
toolbox.register("attr_float", rnd.uniform, -15, 15)
toolbox.register(
    "individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=gen_size
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

We add our landscape to the toolset, indicate that the mating will use a uniform crossover on a per-gene basis, that the mutation will be also on a per-gene basis with a value taken from a gaussian distribution, and that parent and survivor selections will use tournament selection.

In [14]:
toolbox.register("evaluate", evaluate)

toolbox.register("mate", tools.cxUniform)
toolbox.register("mutate", tools.mutGaussian, mu=0)

toolbox.register("par_select", tools.selTournament)
toolbox.register("sur_select", tools.selTournament)

We define that for each generation we'll summarize the fitnesses with median, mean, standard deviation, and store the best and worst fitnesses in the generation.

In [15]:
stat = tools.Statistics(key=lambda ind: ind.fitness.values[0])

stat.register("med", stats.median)
stat.register("avg", stats.mean)
stat.register("std", stats.stdev)
stat.register("best", min)
stat.register("worst", max)

We invoque the data recording logbook.

In [16]:
logbook = tools.Logbook()

## Experiment description

64 Experiments
>L-> In each experiment, one set of parameters is used.
>>L-> 40 Replicates per experiment.
>>>L-> Each replicate is different due to randomness effects.

# Running the experiments

## Combining the 2-level Factors

We create a list with all the possible combinations of the 2-level factors

In [17]:
exp_par = list(
    it.product(
        pop_size_lvl,
        b_ratio_lvl,
        cx_pb_lvl,
        mut_pb_lvl,
        mut_sig_lvl,
        k_par_lvl,
        k_sur_lvl,
    )
)
print('Cantidad de combinaciones de parametros en "exp_par" :' + str(len(exp_par)))
print()
print('Primera y última combinación de parametros en "exp_par":')
print("Secuencia (pop, b_ratio, cx_pb, mut_pb, mut_sig, k_par, k_sur)")
print(exp_par[0])
print(exp_par[-1])

Cantidad de combinaciones de parametros en "exp_par" :128

Primera y última combinación de parametros en "exp_par":
Secuencia (pop, b_ratio, cx_pb, mut_pb, mut_sig, k_par, k_sur)
(20, 0.6, 0.1, 0.1, 0.5, 2, 2)
(160, 5, 0.9, 0.9, 7.5, 6, 6)


### Executing the experiment

In [ ]:
rnd.seed(42)

In [16]:
pickle_file = pickle_dir + file_sufix + ".joblib"

In [18]:
%%time
if __name__ == "__main__":
    for (pop_size, b_ratio, cx_pb, mut_pb, mut_sig, k_par, k_sur), j in zip(
        exp_par, range(len(exp_par))
    ):
        exp_n = j + 1
        par_size = b_ratio * pop_size
        for rep_n in range(rep_end):
            rep_seed = rnd.randint(0, 999)
            rnd.seed(rep_seed)
            # We initialize the population and evaluate the individuals' fitnesses
            pop = toolbox.population(n=pop_size)
            fitnesses = toolbox.map(toolbox.evaluate, pop)
            for ind, fit in zip(pop, fitnesses):
                ind.fitness.values = fit
            # We start the logbook
            record = stat.compile(pop)
            births = len(pop)
            logbook.record(
                exp=exp_n,
                pop=pop_size,
                b_ratio=b_ratio,
                cx_pb=cx_pb,
                mut_pb=mut_pb,
                mut_sig=mut_sig,
                k_par=k_par,
                k_sur=k_sur,
                rep=rep_n + 1,
                seed=rep_seed,
                births=births,
                **record
            )

            while births < births_end:
                # Select Parents and clone them as base for offsprings
                parents = toolbox.par_select(pop, k=par_size, tournsize=k_par)
                offspring = [toolbox.clone(ind) for ind in parents]
                births = births + len(offspring)

                # Aplly crossover
                for child1, child2 in zip(offspring[::2], offspring[1::2]):
                    toolbox.mate(child1, child2, indpb=cx_pb)

                # Apply mutation
                for mutant in offspring:
                    toolbox.mutate(mutant, sigma=mut_sig, indpb=mut_pb)
                    del mutant.fitness.values

                fitnesses = toolbox.map(toolbox.evaluate, offspring)
                for ind, fit in zip(offspring, fitnesses):
                    ind.fitness.values = fit

                pop = toolbox.sur_select((pop + offspring), k=pop_size, tournsize=k_sur)

                record = stat.compile(pop)
                logbook.record(
                    exp=exp_n,
                    pop=pop_size,
                    b_ratio=b_ratio,
                    cx_pb=cx_pb,
                    mut_pb=mut_pb,
                    mut_sig=mut_sig,
                    k_par=k_par,
                    k_sur=k_sur,
                    rep=rep_n + 1,
                    seed=rep_seed,
                    births=births,
                    **record
                )
    if j % 8 == 0:
        with open(pickle_file, "wb") as handle:
            joblib.dump(logbook, handle, compress=("xz", 2))

Wall time: 14min 25s


Storing the logbook to a file

In [39]:
%%time
with open(pickle_file, "wb") as handle:
    joblib.dump(logbook, handle, compress=("xz", 2))

# Data Storage

Reading the logbook from the file

In [ ]:
pickle_file = pickle_dir + file_sufix + ".joblib"

In [7]:
%%time
with open(pickle_file, "rb") as handle:
    logbook = joblib.load(handle)

CPU times: user 3min 28s, sys: 7.81 s, total: 3min 36s
Wall time: 3min 36s


## Storing DataFrame of the Logbook

We transform the records into a data frame

In [8]:
%%time
pop_records = [record for record in logbook]
fitness_res = pd.DataFrame.from_dict(pop_records)

CPU times: user 13.9 s, sys: 1.49 s, total: 15.4 s
Wall time: 15.4 s


In [9]:
cols = [
    "exp",
    "pop",
    "b_ratio",
    "cx_pb",
    "mut_pb",
    "mut_sig",
    "k_par",
    "k_sur",
    "rep",
    "seed",
    "births",
    "avg",
    "best",
    "med",
    "std",
    "worst",
]
fitness_res = fitness_res[cols]
fitness_res.head()

,exp,pop,b_ratio,cx_pb,mut_pb,mut_sig,k_par,k_sur,rep,seed,births,avg,best,med,std,worst
0,1,20,3.0,0.5,0.5,2.5,2,6,1,216,20,53.166835,19.791813,51.714896,13.730201,75.854941
1,1,20,3.0,0.5,0.5,2.5,2,6,1,216,80,34.083972,9.342030,39.407308,13.170811,51.182807
2,1,20,3.0,0.5,0.5,2.5,2,6,1,216,140,14.236409,4.324023,9.342030,8.896384,39.407308
3,1,20,3.0,0.5,0.5,2.5,2,6,1,216,200,7.788521,3.067776,7.975604,2.831520,15.255258
4,1,20,3.0,0.5,0.5,2.5,2,6,1,216,260,5.960772,1.564706,7.729387,2.519881,8.972362


We store the DataFrame in an external file

In [10]:
fit_log_df_file = pickle_dir + file_sufix + "_fit_log_df.xz"
fitness_res.to_pickle(fit_log_df_file)
print(len(fitness_res))

3279360


We read them if necessary

In [5]:
fit_log_df_file = pickle_dir + file_sufix + "_fit_log_df.xz"
fitness_res = pd.read_pickle(fit_log_df_file)
print(len(fitness_res))

3279360


## Storing DataFrame of ca. 400 samples of each generation

In [ ]:
%%time
index = [
    "exp",
    "pop",
    "b_ratio",
    "cx_pb",
    "mut_pb",
    "mut_sig",
    "k_par",
    "k_sur",
    "rep",
    "seed",
]
print(fitness_res.groupby(index).size())

In [12]:
def filter_group(dfg, col, qty):
    col_min = dfg[col].min()
    col_max = dfg[col].max()
    col_length = dfg[col].size
    jumps = col_length - 1
    jump_size = int((col_max - col_min) / jumps)
    new_jump_size = jumps / qty
    if new_jump_size > 1:
        new_jump_size = int(new_jump_size) * jump_size
    else:
        new_jump_size = jump_size

    col_select = list(range(col_min, col_max, new_jump_size))
    col_select.append(col_max)

    return dfg[dfg[col].isin(col_select)]

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10 µs


In [ ]:
%%time
index = [
    "exp",
    "pop",
    "b_ratio",
    "cx_pb",
    "mut_pb",
    "mut_sig",
    "k_par",
    "k_sur",
    "rep",
    "seed",
]
grouped = fitness_res.groupby(index, group_keys=False).apply(
    lambda x: filter_group(x, "births", 400)
)
print(grouped.groupby(index).size())

In [14]:
%%time
fit_df_file = pickle_dir + file_sufix + "_fit_df.xz"
grouped.to_pickle(fit_df_file)
print(len(grouped))

582440
CPU times: user 11.1 s, sys: 19.9 ms, total: 11.1 s
Wall time: 11.5 s


## Storing DataFrame of final values of each replicate

We filter the final values of each replicate and store the Data Frame

In [15]:
%%time
query = fitness_res["births"] > births_end
fit_fin_res = fitness_res[query]

In [15]:
fit_fin_df_file = pickle_dir + file_sufix + "_fit_fin_df.xz"
fit_fin_res.to_pickle(fit_fin_df_file)
print(len(fit_fin_res))

1400
CPU times: user 48.8 ms, sys: 992 µs, total: 49.8 ms
Wall time: 257 ms


## Storing DataFrame of values after birth 30K

In [17]:
%%time
query = fitness_res["births"] > 30e3
fit_30k_res = fitness_res[query]

print(len(fit_30k_res))

1640400
CPU times: user 870 ms, sys: 6.83 ms, total: 877 ms
Wall time: 878 ms


In [18]:
index = [
    "exp",
    "pop",
    "b_ratio",
    "cx_pb",
    "mut_pb",
    "mut_sig",
    "k_par",
    "k_sur",
    "rep",
    "seed",
]
fit_30k_res = fit_30k_res.sort_values("births").groupby(index, as_index=False).first()

In [18]:
fit_30k_df_file = pickle_dir + file_sufix + "_fit_30k_df.xz"
fit_30k_res.to_pickle(fit_30k_df_file)
len(fit_30k_res)

1440